In [0]:
configs ={"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id":"{Client ID}",
          "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope="{keyvault name}",key="{client secret name}"),
          "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/{Directory ID}/oauth2/token" }
try:
    dbutils.fs.mount(
    source="abfss://data@{Storage Account Name}.dfs.core.windows.net/",
    mount_point="/mnt/data/",
    extra_configs = configs)
except Exception as e:
    print ("Error: {} already mounted.Run unmount first")

In [0]:
import dlt
from pyspark.sql.functions import *
from pyspark.sql.types import *
import datetime

#create streaming delta live table
salesorderstreampath ='/mnt/data/'
df = spark.read.json(salesorderstreampath)
@dlt.table(
    comment="the streaming raw dataset."
    )
    
def sales_orders_stream_raw():
    return spark.readStream.schema(df.schema).json(salesorderstreampath)


In [0]:
# customers raw delta live table 
customerpath='/mnt/data/customers/customers.parquet'
@dlt.table(
    comment="the customers raw dataset."
    )
def customers_raw():
    return (spark.read.parquet(customerpath,header=True))

In [0]:
#products raw delta live table
productpath ='/mnt/data/products/products.parquet' 
@dlt.table(
    comment="the prdduct raw dataset."
    )
def products_raw():
    return (spark.read.parquet(productpath,header=True))

In [0]:
# sales order batch raw delta live table
salesorderbatchpath='/mnt/data/sales_orders/sales_orders.parquet' 
@dlt.table(
    comment="the sales order batch raw dataset.."
    )
def sales_orders_batch_raw():
    return (spark.read.parquet(salesorderbatchpath,header=True))

In [0]:
    sales_orders_schema = StructType(
    [
        StructField("customer_id", LongType(), True),
        StructField("customer_name", StringType(), True),
        StructField("order_datetime", StringType(), True),
        StructField("order_number", LongType(), True),
        StructField(
            "ordered_products",
            ArrayType(
                StructType(
                    [
                        StructField("curr", StringType(), True),
                        StructField("id", StringType(), True),
                        StructField("name", StringType(), True),
                        StructField("price", IntegerType(), True),
                        StructField("qty", IntegerType(), True),
                        StructField("unit", StringType(), True),
                        StructField(
                            "promotion_info",
                            StructType(
                                [
                                    StructField("promo_id", IntegerType(), True),
                                    StructField("promo_qty", IntegerType(), True),
                                    StructField("promo_disc", DecimalType(3, 2), True),
                                    StructField("promo_item", StringType(), True),
                                ]
                            ),
                            True,
                        ),
                    ]
                ),
                True,
            ),
            True,
        ),
        StructField("number_of_line_items", LongType(), True),
        StructField(
            "clicked_items", ArrayType(ArrayType(StringType(), True), True), True
        ),
        StructField(
            "promo_info",
            ArrayType(
                StructType(
                    [
                        StructField("promo_id", IntegerType(), True),
                        StructField("promo_qty", IntegerType(), True),
                        StructField("promo_disc", DecimalType(3, 2), True),
                        StructField("promo_item", StringType(), True),
                    ]
                ),
                True,
            ),
            True,
        ),
    ]
) 
@dlt.table(
    comment="Load data to sales_orders cleansed table",
    table_properties={"pipelines.reset.allowed": "true"},
    spark_conf={"pipelines.trigger.interval": "60 seconds"},
    temporary=False,
)
def sales_orders_cleansed():
    return (
        dlt.readStream("sales_orders_stream_raw")        
        .select(get_json_object(to_json(col("payload")), "$.after").alias("row"))
        .withColumn("row", regexp_replace("row", '"\\[', "["))
        .withColumn("row", regexp_replace("row", '\\]"', "]"))
        .withColumn("row", regexp_replace("row", "\\\\", ""))
        .select(from_json(col("row"), sales_orders_schema).alias("row"))
        .select("row.*")
        .withColumn("ordered_products", explode("ordered_products"))
        .withColumn("order_datetime", from_unixtime("order_datetime"))
        .withColumn("product_id", col("ordered_products").id)
        .withColumn("unit_price", col("ordered_products").price)
        .withColumn("quantity", col("ordered_products").qty)
    )

In [0]:
   schema = ArrayType(
    StructType(
        [
            StructField("qty", IntegerType(), True),
            StructField("unit", StringType(), True),
            StructField("curr", StringType(), True),
            StructField("id", StringType(), True),
            StructField("name", StringType(), True),
            StructField("price", IntegerType(), True),
            StructField(
                "promotion_info",
                StructType(
                    [
                        StructField("promo_id", IntegerType(), True),
                        StructField("promo_qty", IntegerType(), True),
                        StructField("promo_disc", DecimalType(3, 2), True),
                        StructField("promo_item", StringType(), True),
                    ]
                ),
                True,
            ),
        ]
    )
)

schema2 = ArrayType(
    StructType(
        [
            StructField("promo_id", IntegerType(), True),
            StructField("promo_qty", IntegerType(), True),
            StructField("promo_disc", DecimalType(3, 2), True),
            StructField("promo_item", StringType(), True),
        ]
    ),
    True,
)
schema3 = ArrayType(ArrayType(StringType(), True), True)

@dlt.table(
    comment="Load data to customers cleansed table",
    table_properties={"pipelines.reset.allowed": "true"},
    spark_conf={"pipelines.trigger.interval": "60 seconds"},
    temporary=False,
)

def sales_orders_batch_cleansed():
    return (
        dlt.read("sales_orders_batch_raw")
        .select("*")
        .withColumn("clicked_items", from_json(col("clicked_items"), schema3))
        .withColumn("promo_info", from_json(col("promo_info"), schema2))
        .withColumn("ordered_products", from_json(col("ordered_products"), schema))
        .withColumn("ordered_products", explode("ordered_products"))
        .withColumn("order_datetime", from_unixtime("order_datetime"))
        .withColumn("product_id", col("ordered_products").id)
        .withColumn("unit_price", col("ordered_products").price)
        .withColumn("quantity", col("ordered_products").qty)
    )
    
def customers_cleansed():
    return (
        dlt.read("customers_raw")
    )

In [0]:
   @dlt.table(
    comment="Load data to a products cleansed table",
    table_properties={"pipelines.reset.allowed": "true"},
    spark_conf={"pipelines.trigger.interval": "60 seconds"},
    temporary=False,
)

In [0]:
   import dlt
from pyspark.sql.functions import col, expr

@dlt.view(spark_conf={"pipelines.incompatibleViewCheck.enabled": "false"})
def users():
  return spark.readStream.format("delta").option("ignoreChanges", "true").table("retail_org.sales_orders_batch_cleansed")

dlt.create_streaming_live_table("sales_orders_cleansed")

dlt.apply_changes(
  target = "sales_orders_cleansed",
  source = "users",
  keys = ["order_number"],
  sequence_by = col("order_datetime"),
  stored_as_scd_type = 1
)

In [0]:
    @dlt.table(
    schema="""
         product_key BIGINT GENERATED ALWAYS AS identity,
         product_id STRING,
         product_category STRING,
         product_name STRING,
         sales_price STRING,
         ean13 DOUBLE,
         ean5 STRING,
         product_unit STRING    
    """,
    comment="Load data to products dimension table",
    table_properties={"quality": "gold", "pipelines.reset.allowed": "true"},
    spark_conf={"pipelines.trigger.interval": "60 seconds"},
    temporary=False,
)
def dim_products():
    return dlt.read("products_cleansed")

In [0]:
@dlt.table(
    schema="""
          customer_key BIGINT GENERATED ALWAYS AS IDENTITY,
          customer_id integer,
          tax_id STRING,
          tax_code STRING,
          customer_name STRING,
          state STRING,
          city STRING,
          postcode STRING,
          street STRING,
          number STRING,
          unit STRING,
          region STRING,
          district STRING,
          lon double,
          lat double,
          ship_to_address STRING,
          valid_from STRING,
          valid_to STRING,
          units_purchased STRING,
          loyalty_segment integer
    """,
    comment="Load data to customers dimension table",
    table_properties={"quality": "gold", "pipelines.reset.allowed": "true"},
    spark_conf={"pipelines.trigger.interval": "60 seconds"},
    temporary=False,
)
def dim_customers():
    return dlt.read("customers_cleansed")

In [0]:
@dlt.table(
    comment="load data to sales orders fact table",
    table_properties={"quality": "gold", "pipelines.reset.allowed": "true"},
    spark_conf={"pipelines.trigger.interval": "60 seconds"},
    temporary=False,
)
def fact_sales_orders():
    s = dlt.read("sales_orders_cleansed").alias("s")
    p = dlt.read("dim_products").alias("p")
    c = dlt.read("dim_customers").alias("c")
    return (
        s.join(p, s.product_id == p.product_id, "inner")
        .join(c, s.customer_id == c.customer_id, "inner")
        .select(
            "s.order_number",
            "c.customer_key",
            "p.product_key",
            col("s.order_datetime").cast("date").alias("order_date"),
            "s.unit_price",
            "s.quantity",
            expr("s.unit_price * s.quantity").alias("total_price"),
        )
    )

In [0]:
@dlt.table(
    comment="load data to customer sales fact table",
    table_properties={"quality": "gold", "pipelines.reset.allowed": "true"},
    spark_conf={"pipelines.trigger.interval": "60 seconds"},
    temporary=False,
)
def fact_customer_sales():
    s = dlt.read("sales_orders_cleansed").alias("s")
    p = dlt.read("dim_products").alias("p")
    c = dlt.read("dim_customers").alias("c")
    return (
        s.join(p, s.product_id == p.product_id, "inner")
        .join(c, s.customer_id == c.customer_id, "inner")
        .groupBy("c.customer_key", "p.product_key")
        .agg(
            sum("quantity").alias("total_quantity"),
            sum(expr("s.unit_price * s.quantity")).alias("sale_amount"),
        )
    )